In [3]:
import gym
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.autograd import Variable

# One hot encoding function
def to_one_hot(i, n_classes=None):
    a = np.zeros(n_classes, 'uint8')
    a[i] = 1
    return a

# Neural Network Model
class Network(nn.Module):
    def __init__(self, state_size: int, action_size: int):
        super(Network, self).__init__()
        self.layer1 = nn.Linear(state_size, hidden_size)
        self.layer2 = nn.Linear(hidden_size, hidden_size)
        self.value = nn.Linear(hidden_size, action_size)

    def forward(self, state):
        state = torch.from_numpy(state).float() if not isinstance(state, torch.Tensor) else state
        layer1 = torch.relu(self.layer1(state))
        layer2 = torch.relu(self.layer2(layer1))
        value = self.value(layer2)
        return value

class DQNAgent:
    def __init__(self, env: gym.Env):
        self.env = env
        self.state_size = env.observation_space.n
        self.action_size = env.action_space.n
        self.lr = 0.001
        self.gamma = 0.99
        self.dqn = Network(self.state_size, self.action_size)
        self.optimizer = optim.Adam(self.dqn.parameters(), lr=self.lr)

    def get_action(self, state, epsilon):
        state = to_one_hot(state, self.state_size)
        state = torch.from_numpy(state).float().unsqueeze(0)
        q_value = self.dqn(state)[0]
        if np.random.rand() <= epsilon:
            return np.random.choice(self.action_size)
        else:
            return torch.argmax(q_value).item()

    def train_step(self, state, action, reward, next_state, done, next_action):
        state = to_one_hot(state, self.state_size)
        next_state = to_one_hot(next_state, self.state_size)
        state = torch.from_numpy(state).float()
        next_state = torch.from_numpy(next_state).float()

        curr_Q = self.dqn(state)[action]
        next_Q = self.dqn(next_state).detach()

        target_Q = torch.tensor([reward + self.gamma * next_Q[next_action]]) if not done else torch.tensor([reward])

        loss = torch.nn.functional.mse_loss(curr_Q, target_Q)

        self.optimizer.zero_grad()
        loss.backward()
        self.optimizer.step()



In [4]:
import gym
import numpy as np
import torch
from tqdm import tqdm

# PyTorch 기반의 DQNAgent 클래스 및 필요한 추가 모듈 정의는 여기에 추가

# 2.2 환경 생성
env_name = "FrozenLake-v1"
env = gym.make(env_name)
env.seed(1)  # 재현 가능성을 위해

# 2.4 하이퍼파라미터 초기화
hidden_size = 128
max_episodes = 2500  # 총 에피소드 수

# 탐험 파라미터
epsilon = 1.0        # 탐험률
max_epsilon = 1.0    # 시작시 탐험 확률
min_epsilon = 0.01   # 최소 탐험 확률
decay_rate = 0.005   # 탐험 확률의 지수 감소율

# 에이전트 훈련
agent = DQNAgent(env)

if __name__ == "__main__":
    scores = []

    with tqdm(total=max_episodes, desc="에피소드 진행") as pbar:
        for episode in range(max_episodes):
            state = agent.env.reset()
            episode_reward = 0
            done = False

            while not done:
                action = agent.get_action(state, epsilon)
                next_state, reward, done, _ = agent.env.step(action)
                next_action = agent.get_action(next_state, epsilon)

                agent.train_step(state, action, reward, next_state, done, next_action)

                state = next_state
                episode_reward += reward

                if done:
                    scores.append(episode_reward)
                    pbar.set_postfix({'episode_reward': episode_reward})
                    pbar.update(1)
                    break

            epsilon = min_epsilon + (max_epsilon - min_epsilon) * np.exp(-decay_rate * episode)

    print(f"평균 점수: {sum(scores) / max_episodes}")

    count = 500
    rewards_per_thousand_episodes = np.split(np.array(scores), int(max_episodes / 500))

    print("********천 개 에피소드당 평균 보상********\n")
    for r in rewards_per_thousand_episodes:
        print(f"{count}: {sum(r) / 500}")
        count += 500


/usr/local/lib/python3.10/dist-packages/gym/core.py:317: DeprecationWarning: WARN: Initializing wrapper in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(
/usr/local/lib/python3.10/dist-packages/gym/wrappers/step_api_compatibility.py:39: DeprecationWarning: WARN: Initializing environment in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(
/usr/local/lib/python3.10/dist-packages/gym/core.py:256: DeprecationWarning: WARN: Function `env.seed(seed)` is marked as deprecated and will be removed in the future. Please use `env.reset(seed=seed)` instead.
  deprecation(
에피소드 진행:   0%|          | 0/2500 [00:00<?, ?it/s]<ipython-input-3-a3a48a4116ac>:59: UserWarning: Using a target size (torch.Size([1])) that is different to the input size (

평균 점수: 0.3584
********천 개 에피소드당 평균 보상********

500: 0.008
1000: 0.058
1500: 0.518
2000: 0.594
2500: 0.614
